In [ ]:
!pip install transformers[torch]
!pip install accelerate -U
!pip install datasets

# 构建一个GitHub Issue标注程序

In [ ]:
import pandas as pd
dataset_url = "https://git.io/nlp-with-transformers"
df_issues = pd.read_json(dataset_url, lines=True)
print(f"DataFrame shape: {df_issues.shape}")
cols = ["url", "id", "title", "user", "labels", "state", "created_at", "body"]

df_issues.loc[2, cols].to_frame()


In [ ]:
[
    {
        "id":2659267025, "node_id":"MDU6TGFiZWwyNjU5MjY3MDI1",
        "url":"https://api.github.com/repos/huggingface...",
        "name":"DeepSpeed",
        "color":"4D34F7",
        "default":false,
        "description":""
    }
]


In [ ]:
df_issues["labels"] = (df_issues["labels"] .apply(lambda x: [meta["name"] for meta in x]))
df_issues[["labels"]].head()


In [ ]:
df_issues["labels"].apply(lambda x : len(x)).value_counts().to_frame().T


In [ ]:
df_counts = df_issues["labels"].explode().value_counts()
print(f"Number of labels: {len(df_counts)}")

In [ ]:
label_map = {"Core: Tokenization": "tokenization", "New model": "new model", "Core: Modeling": "model training", "Usage": "usage", "Core: Pipeline": "pipeline", "TensorFlow": "tensorflow or tf", "PyTorch": "pytorch", "Examples": "examples", "Documentation": "documentation"}

def filter_labels(x):
	return [label_map[label] for label in x if label in label_map]

df_issues["labels"] = df_issues["labels"].apply(filter_labels)
all_labels = list(label_map.values())


In [ ]:
df_counts = df_issues["labels"].explode().value_counts()
df_counts.to_frame().T

In [ ]:
df_issues["split"] = "unlabeled"
mask = df_issues["labels"].apply(lambda x: len(x)) > 0
df_issues.loc[mask, "split"] = "labeled"
df_issues["split"].value_counts().to_frame()


In [ ]:
for column in ["title", "body", "labels"]:
	print(f"{column}: {df_issues[column].iloc[26][:500]}\n")

In [ ]:
df_issues["text"] = (df_issues .apply(lambda x: x["title"] + "\n\n" + x["body"], axis=1))
len_before = len(df_issues)
df_issues = df_issues.drop_duplicates(subset="text")
print(f"Removed {(len_before-len(df_issues))/len_before:.2%} duplicates.")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

(df_issues["text"].str.split().apply(len) .hist(bins=np.linspace(0, 500, 50), grid=False, edgecolor="C0"))
plt.title("Words per issue")
plt.xlabel("Number of words")
plt.ylabel("Number of issues")
plt.show()


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit([all_labels])
mlb.transform([["tokenization", "new model"], ["pytorch"]])

In [ ]:
from skmultilearn.model_selection import iterative_train_test_split
def balanced_split(df, test_size=0.5):
	ind = np.expand_dims(np.arange(len(df)), axis=1)
	labels = mlb.transform(df["labels"])
	ind_train, _, ind_test, _ = iterative_train_test_split(ind, labels, test_size)
    return df.iloc[ind_train[:, 0]], df.iloc[ind_test[:,0]]


In [ ]:
from sklearn.model_selection import train_test_split
df_clean = df_issues[["text", "labels", "split"]].reset_index(drop=True).copy()
df_unsup = df_clean.loc[df_clean["split"] == "unlabeled", ["text", "labels"]]
df_sup = df_clean.loc[df_clean["split"] == "labeled", ["text", "labels"]]

np.random.seed(0)
df_train, df_tmp = balanced_split(df_sup, test_size=0.5)
df_valid, df_test = balanced_split(df_tmp, test_size=0.5)


In [ ]:
from datasets import Dataset, DatasetDict

ds = DatasetDict({ "train": Dataset.from_pandas(df_train.reset_index(drop=True)), "valid": Dataset.from_pandas(df_valid.reset_index(drop=True)), "test": Dataset.from_pandas(df_test.reset_index(drop=True)), "unsup": Dataset.from_pandas(df_unsup.reset_index(drop=True))})


In [ ]:
np.random.seed(0)
all_indices = np.expand_dims(list(range(len(ds["train"]))), axis=1)
indices_pool = all_indices
labels = mlb.transform(ds["train"]["labels"])
train_samples = [8, 16, 32, 64, 128]
train_slices, last_k = [], 0
for i, k in enumerate(train_samples):
	# Split off samples necessary to fill the gap to the next split size
	indices_pool, labels, new_slice, _ = iterative_train_test_split( indices_pool, labels, (k-last_k)/len(labels))
	last_k = k
	if i==0:
		train_slices.append(new_slice)
	else:
		train_slices.append(np.concatenate((train_slices[-1], new_slice)))
# Add full dataset as last slice
train_slices.append(all_indices), train_samples.append(len(ds["train"]))
train_slices = [np.squeeze(train_slice) for train_slice in train_slices]


In [ ]:
print("Target split sizes:")
print(train_samples)
print("Actual split sizes:")
print([len(x) for x in train_slices])


# 实现朴素贝叶斯基线

In [ ]:
def prepare_labels(batch):
	batch["label_ids"] = mlb.transform(batch["labels"])
	return batch
ds = ds.map(prepare_labels, batched=True)
from collections import defaultdict

macro_scores, micro_scores = defaultdict(list), defaultdict(list)


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.feature_extraction.text import CountVectorizer
for train_slice in train_slices:
	# Get training slice and test data
	ds_train_sample = ds["train"].select(train_slice)
	y_train = np.array(ds_train_sample["label_ids"])
	y_test = np.array(ds["test"]["label_ids"])
	# Use a simple count vectorizer to encode our texts as token counts
	count_vect = CountVectorizer()
	X_train_counts = count_vect.fit_transform(ds_train_sample["text"])
	X_test_counts = count_vect.transform(ds["test"]["text"])
	# Create and train our model!
	classifier = BinaryRelevance(classifier=MultinomialNB()) classifier.fit(X_train_counts, y_train)
	# Generate predictions and evaluate
	y_pred_test = classifier.predict(X_test_counts)
	clf_report = classification_report( y_test, y_pred_test, target_names=mlb.classes_, zero_division=0, output_dict=True)
	# Store metrics
	macro_scores["Naive Bayes"].append(clf_report["macro avg"]["f1-score"])
	micro_scores["Naive Bayes"].append(clf_report["micro avg"]["f1-score"])


In [ ]:
import matplotlib.pyplot as plt
def plot_metrics(micro_scores, macro_scores, sample_sizes, current_model):
	fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(10, 4), sharey=True)
	for run in micro_scores.keys():
		if run == current_model:
			ax0.plot(sample_sizes, micro_scores[run], label=run, linewidth=2)
			ax1.plot(sample_sizes, macro_scores[run], label=run, linewidth=2)
		else:
			ax0.plot(sample_sizes, micro_scores[run], label=run, linestyle="dashed")
			ax1.plot(sample_sizes, macro_scores[run], label=run, linestyle="dashed")
	ax0.set_title("Micro F1 scores")
	ax1.set_title("Macro F1 scores")
	ax0.set_ylabel("Test set F1 score")
	ax0.legend(loc="lower right")
	for ax in [ax0, ax1]:
		ax.set_xlabel("Number of training samples")
		ax.set_xscale("log")
		ax.set_xticks(sample_sizes)
		ax.set_xticklabels(sample_sizes)
		ax.minorticks_off()
	plt.tight_layout()
	plt.show()
plot_metrics(micro_scores, macro_scores, train_samples, "Naive Bayes")


# 零样本学习

In [ ]:
from transformers import pipeline
pipe = pipeline("fill-mask", model="bert-base-uncased")
movie_desc = "The main characters of the movie madacascar \ are a lion, a zebra, a giraffe, and a hippo. "
prompt = "The movie is about [MASK]."
output = pipe(movie_desc + prompt)
for element in output:
	print(f"Token {element['token_str']}:\t{element['score']:.3f}%")


In [ ]:
output = pipe(movie_desc + prompt, targets=["animals", "cars"])
for element in output:
	print(f"Token {element['token_str']}:\t{element['score']:.3f}%")


In [ ]:
movie_desc = "In the movie transformers aliens \ can morph into a wide range of vehicles."
output = pipe(movie_desc + prompt, targets=["animals", "cars"])
for element in output:
	print(f"Token {element['token_str']}:\t{element['score']:.3f}%")


In [ ]:
from transformers import pipeline
pipe = pipeline("zero-shot-classification", device=0)
sample = ds["train"][0]
print(f"Labels: {sample['labels']}")
output = pipe(sample["text"], all_labels, multi_label=True)
print(output["sequence"][:400])
print("\nPredictions:")
for label, score in zip(output["labels"], output["scores"]):
	print(f"{label}, {score:.2f}")

In [ ]:
def zero_shot_pipeline(example):
	output = pipe(example["text"], all_labels, multi_label=True)
	example["predicted_labels"] = output["labels"]
	example["scores"] = output["scores"]
	return example
ds_zero_shot = ds["valid"].map(zero_shot_pipeline)


In [ ]:
def get_preds(example, threshold=None, topk=None):
	preds = []
	if threshold:
		for label, score in zip(example["predicted_labels"], example["scores"]):
			if score >= threshold:
				preds.append(label)
	elif topk:
		for i in range(topk):
            preds.append(example["predicted_labels"][i])
    else:
         raise ValueError("Set either `threshold` or `topk`.")
    return {"pred_label_ids": list(np.squeeze(mlb.transform([preds])))}


In [ ]:
def get_clf_report(ds):
	y_true = np.array(ds["label_ids"])
	y_pred = np.array(ds["pred_label_ids"])
	return classification_report( y_true, y_pred, target_names=mlb.classes_, zero_division=0, output_dict=True)


In [ ]:
macros, micros = [], []
topks = [1, 2, 3, 4]
for topk in topks:
	ds_zero_shot = ds_zero_shot.map(get_preds, batched=False, fn_kwargs={'topk': topk})
	clf_report = get_clf_report(ds_zero_shot)
	micros.append(clf_report['micro avg']['f1-score'])
	macros.append(clf_report['macro avg']['f1-score'])
plt.plot(topks, micros, label='Micro F1')
plt.plot(topks, macros, label='Macro F1')
plt.xlabel("Top-k")
plt.ylabel("F1-score")
plt.legend(loc='best')
plt.show()


In [ ]:
macros, micros = [], []
thresholds = np.linspace(0.01, 1, 100)
for threshold in thresholds:
	ds_zero_shot = ds_zero_shot.map(get_preds, fn_kwargs={"threshold": threshold})
	clf_report = get_clf_report(ds_zero_shot)
    micros.append(clf_report["micro avg"]["f1-score"])
    macros.append(clf_report["macro avg"]["f1-score"])
plt.plot(thresholds, micros, label="Micro F1")
plt.plot(thresholds, macros, label="Macro F1")
plt.xlabel("Threshold")
plt.ylabel("F1-score")
plt.legend(loc="best")
plt.show()


In [ ]:
best_t, best_micro = thresholds[np.argmax(micros)], np.max(micros)
print(f'Best threshold (micro): {best_t} with F1-score {best_micro:.2f}.') best_t, best_macro = thresholds[np.argmax(macros)], np.max(macros)
print(f'Best threshold (micro): {best_t} with F1-score {best_macro:.2f}.')


In [ ]:
ds_zero_shot = ds['test'].map(zero_shot_pipeline)
ds_zero_shot = ds_zero_shot.map(get_preds, fn_kwargs={'topk': 1})
clf_report = get_clf_report(ds_zero_shot)
for train_slice in train_slices:
	macro_scores['Zero Shot'].append(clf_report['macro avg']['f1-score'])
	micro_scores['Zero Shot'].append(clf_report['micro avg']['f1-score'])
plot_metrics(micro_scores, macro_scores, train_samples, "Zero Shot")


# 少样本学习

In [ ]:
from transformers import set_seed
import nlpaug.augmenter.word as naw

set_seed(3)
aug = naw.ContextualWordEmbsAug(model_path="distilbert-base-uncased", device="cpu", action="substitute")
text = "Transformers are the most popular toys"
print(f"Original text: {text}")
print(f"Augmented text: {aug.augment(text)}")

In [ ]:
def augment_text(batch, transformations_per_example=1):
	text_aug, label_ids = [], []
	for text, labels in zip(batch["text"], batch["label_ids"]):
		text_aug += [text]
		label_ids += [labels]
		for _ in range(transformations_per_example):
			text_aug += [aug.augment(text)]
			label_ids += [labels]
    return {"text": text_aug, "label_ids": label_ids}
ds_train_sample = ds_train_sample.map(augment_text, batched=True, remove_columns=ds_train_sample.column_names).shuffle(seed=42)
plot_metrics(micro_scores, macro_scores, train_samples, "Naive Bayes + Aug")


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
model_ckpt = "miguelvictor/python-gpt2-large"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)
def mean_pooling(model_output, attention_mask):
	# Extract the token embeddings
	token_embeddings = model_output[0]
	# Compute the attention mask
	input_mask_expanded = (attention_mask .unsqueeze(-1) .expand(token_embeddings.size()) .float())
	# Sum the embeddings, but ignore masked tokens
	sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
	sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
	# Return the average as a single vector
	return sum_embeddings / sum_mask
def embed_text(examples):
	inputs = tokenizer(examples["text"], padding=True, truncation=True, max_length=128, return_tensors="pt")
	with torch.no_grad():
		model_output = model(**inputs)
	pooled_embeds = mean_pooling(model_output, inputs["attention_mask"])
	return {"embedding": pooled_embeds.cpu().numpy()}

tokenizer.pad_token = tokenizer.eos_token
embs_train = ds["train"].map(embed_text, batched=True, batch_size=16)
embs_valid = ds["valid"].map(embed_text, batched=True, batch_size=16)
embs_test = ds["test"].map(embed_text, batched=True, batch_size=16)
embs_train.add_faiss_index("embedding")


In [ ]:
i, k = 0, 3
# Select the first query and 3 nearest neighbors
rn, nl = "\r\n\r\n", "\n"
# Used to remove newlines in text for compact display
query = np.array(embs_valid[i]["embedding"], dtype=np.float32)
scores, samples = embs_train.get_nearest_examples("embedding", query, k=k)
print(f"QUERY LABELS: {embs_valid[i]['labels']}")
print(f"QUERY TEXT:\n{embs_valid[i]['text'][:200].replace(rn, nl)} [...]\n")
print("="*50)
print(f"Retrieved documents:")
for score, label, text in zip(scores, samples["labels"], samples["text"]):
	print("="*50)
	print(f"TEXT:\n{text[:200].replace(rn, nl)} [...]")
	print(f"SCORE: {score:.2f}")
	print(f"LABELS: {label}")

In [ ]:
def get_sample_preds(sample, m):
	return (np.sum(sample["label_ids"], axis=0) >= m).astype(int)
def find_best_k_m(ds_train, valid_queries, valid_labels, max_k=17):
	max_k = min(len(ds_train), max_k)
	perf_micro = np.zeros((max_k, max_k))
	perf_macro = np.zeros((max_k, max_k))
	for k in range(1, max_k):
		for m in range(1, k + 1):
        	_, samples = ds_train.get_nearest_examples_batch("embedding", valid_queries, k=k)
        	y_pred = np.array([get_sample_preds(s, m) for s in samples])
        	clf_report = classification_report(valid_labels, y_pred
target_names=mlb.classes_, zero_division=0, output_dict=True)
			perf_micro[k, m] = clf_report["micro avg"]["f1-score"]
			perf_macro[k, m] = clf_report["macro avg"]["f1-score"]
	return perf_micro, perf_macro


In [ ]:
valid_labels = np.array(embs_valid["label_ids"])
valid_queries = np.array(embs_valid["embedding"], dtype=np.float32)
perf_micro, perf_macro = find_best_k_m(embs_train, valid_queries, valid_labels)
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(10, 3.5), sharey=True)
ax0.imshow(perf_micro)
ax1.imshow(perf_macro)
ax0.set_title("micro scores")
ax0.set_ylabel("k")
ax1.set_title("macro scores")
for ax in [ax0, ax1]:
	ax.set_xlim([0.5, 17 - 0.5])
	ax.set_ylim([17 - 0.5, 0.5])
	ax.set_xlabel("m")
plt.show()


In [ ]:
k, m = np.unravel_index(perf_micro.argmax(), perf_micro.shape)
print(f"Best k: {k}, best m: {m}")
embs_train.drop_index("embedding")
test_labels = np.array(embs_test["label_ids"])
test_queries = np.array(embs_test["embedding"], dtype=np.float32)
for train_slice in train_slices:
# Create a Faiss index from training slice
	embs_train_tmp = embs_train.select(train_slice)
	embs_train_tmp.add_faiss_index("embedding")
    # Get best k, m values with validation set
    perf_micro, _ = find_best_k_m(embs_train_tmp, valid_queries, valid_labels)
    k, m = np.unravel_index(perf_micro.argmax(), perf_micro.shape)
    # Get predictions on test set
    _, samples = embs_train_tmp.get_nearest_examples_batch("embedding", test_queries, k=int(k))
    y_pred = np.array([get_sample_preds(s, m) for s in samples])
    # Evaluate predictions
    clf_report = classification_report(test_labels, y_pred, target_names=mlb.classes_, zero_division=0, output_dict=True,)
    macro_scores["Embedding"].append(clf_report["macro avg"]["f1-score"])
    micro_scores["Embedding"].append(clf_report["micro avg"]["f1-score"]

plot_metrics(micro_scores, macro_scores, train_samples, "Embedding")



In [ ]:
import torch
from transformers import (AutoTokenizer, AutoConfig, AutoModelForSequenceClassification)

model_ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
def tokenize(batch):
	return tokenizer(batch["text"], truncation=True, max_length=128)
ds_enc = ds.map(tokenize, batched=True)
ds_enc = ds_enc.remove_columns(['labels', 'text'])
ds_enc.set_format("torch")
ds_enc = ds_enc.map(lambda x: {"label_ids_f": x["label_ids"].to(torch.float)}, remove_columns=["label_ids"])
ds_enc = ds_enc.rename_column("label_ids_f", "label_ids")


In [ ]:
from transformers import Trainer, TrainingArguments
training_args_fine_tune = TrainingArguments( output_dir="./results", num_train_epochs=20, learning_rate=3e-5, lr_scheduler_type='constant', per_device_train_batch_size=4, per_device_eval_batch_size=32, weight_decay=0.0, evaluation_strategy="epoch", save_strategy="epoch",logging_strategy="epoch", load_best_model_at_end=True, metric_for_best_model='micro f1', save_total_limit=1, log_level='error')
from scipy.special import expit as sigmoid
def compute_metrics(pred):
	y_true = pred.label_ids
    y_pred = sigmoid(pred.predictions)
    y_pred = (y_pred>0.5).astype(float)
    clf_dict = classification_report(y_true, y_pred, target_names=all_labels, zero_division=0, output_dict=True)
    return {"micro f1": clf_dict["micro avg"]["f1-score"], "macro f1": clf_dict["macro avg"]["f1-score"]}


In [ ]:
config = AutoConfig.from_pretrained(model_ckpt)
config.num_labels = len(all_labels)
config.problem_type = "multi_label_classification"
for train_slice in train_slices:
	model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, config=config)
	trainer = Trainer( model=model, tokenizer=tokenizer, args=training_args_fine_tune, compute_metrics=compute_metrics, train_dataset=ds_enc["train"].select(train_slice), eval_dataset=ds_enc["valid"],)
	trainer.train()
	pred = trainer.predict(ds_enc["test"])
	metrics = compute_metrics(pred)
	macro_scores["Fine-tune (vanilla)"].append(metrics["macro f1"])
	micro_scores["Fine-tune (vanilla)"].append(metrics["micro f1"])

plot_metrics(micro_scores, macro_scores, train_samples, "Fine-tune (vanilla)")


In [ ]:
prompt = """\ Translate English to French: thanks => """
def tokenize(batch):
	return tokenizer(batch["text"], truncation=True, max_length=128, return_special_tokens_mask=True)
	ds_mlm = ds.map(tokenize, batched=True)
	ds_mlm = ds_mlm.remove_columns(["labels", "text", "label_ids"])
from transformers import DataCollatorForLanguageModeling, set_seed
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
set_seed(3)
data_collator.return_tensors = "np"
inputs = tokenizer("Transformers are awesome!", return_tensors="np")
outputs = data_collator([{"input_ids": inputs["input_ids"][0]}])

pd.DataFrame({ "Original tokens": tokenizer.convert_ids_to_tokens(inputs["input_ids"] [0]), "Masked tokens": tokenizer.convert_ids_to_tokens(outputs["input_ids"][0]), "Original input_ids": original_input_ids, "Masked input_ids": masked_input_ids, "Labels": outputs["labels"][0]}).T
data_collator.return_tensors = "pt"
from transformers import AutoModelForMaskedLM

training_args = TrainingArguments( output_dir = f"{model_ckpt}-issues-128", per_device_train_batch_size=32, logging_strategy="epoch", evaluation_strategy="epoch", save_strategy="no", num_train_epochs=16,log_level="error", report_to="none")

trainer = Trainer( model=AutoModelForMaskedLM.from_pretrained("bert-base-uncased"), tokenizer=tokenizer, args=training_args, data_collator=data_collator, train_dataset=ds_mlm["unsup"], eval_dataset=ds_mlm["train"])

trainer.train()


In [ ]:
df_log = pd.DataFrame(trainer.state.log_history)

(df_log.dropna(subset=["eval_loss"]).reset_index()["eval_loss"] .plot(label="Validation"))

df_log.dropna(subset=["loss"]).reset_index()["loss"].plot(label="Train")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(loc="upper right")
plt.show()


In [ ]:
model_ckpt = f'{model_ckpt}-issues-128'
config = AutoConfig.from_pretrained(model_ckpt)
config.num_labels = len(all_labels)
config.problem_type = "multi_label_classification"
for train_slice in train_slices:
	model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, config=config)
	trainer = Trainer( model=model, tokenizer=tokenizer, args=training_args_fine_tune, compute_metrics=compute_metrics
  train_dataset=ds_enc["train"].select(train_slice), eval_dataset=ds_enc["valid"], )
	trainer.train()
	pred = trainer.predict(ds_enc['test'])
	metrics = compute_metrics(pred)
	 # DA refers to domain adaptation
	macro_scores['Fine-tune (DA)'].append(metrics['macro f1'])
	micro_scores['Fine-tune (DA)'].append(metrics['micro f1'])


In [ ]:
plot_metrics(micro_scores, macro_scores, train_samples, "Fine-tune (DA)")
